In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'https://lulacerda.ig.com.br/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('h2')

In [5]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    print(noticia.find_all('a', title=True)[0]['title'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        print(path_image)
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://lulacerda.ig.com.br/anuario-epoca-360-estacio-e-premiada-na-categoria-educacao/
“Anuário Época 360”: Estácio é premiada na categoria Educação
https://lulacerda.ig.com.br/anuario-epoca-360-estacio-e-premiada-na-categoria-educacao/
100% [..............................................................................] 43091 / 43091download.wget
C:\Users\admin\rss3\src\midia_postagem\images\download.wget
['Lu Lacerda']
news_in_db: False
 -- no categories -- 
https://lulacerda.ig.com.br/atelie-benoliel-25-anos-de-muito-sabor/
Ateliê Benoliel: 25 anos de muito sabor
https://lulacerda.ig.com.br/atelie-benoliel-25-anos-de-muito-sabor/
100% [..............................................................................] 47007 / 47007download.wget
C:\Users\admin\rss3\src\midia_postagem\images\download.wget
['Lu Lacerda']
news_in_db: False
 -- no categories -- 
https://lulacerda.ig.com.br/galeria-de-brasileiros-em-ny-chega-ao-rio-em-janeiro/
Galeria de brasileiros em NY chega ao Rio em jan

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://lulacerda.ig.com.br/ddp-diretoria-banda-de-pagode-pop-carioca-comemora-3-anos/
100% [..............................................................................] 43332 / 43332download.wget
C:\Users\admin\rss3\src\midia_postagem\images\download.wget
['Lu Lacerda']
news_in_db: False
 -- no categories -- 
https://lulacerda.ig.com.br/simone-klabin-livro-de-gastronomia-para-colecionadores/
Simone Klabin: livro de gastronomia para colecionadores
https://lulacerda.ig.com.br/simone-klabin-livro-de-gastronomia-para-colecionadores/
100% [..............................................................................] 47379 / 47379download.wget
C:\Users\admin\rss3\src\midia_postagem\images\download.wget
['Lu Lacerda']
news_in_db: False
 -- no categories -- 
https://lulacerda.ig.com.br/le-cordon-bleu-chance-de-bolsa-ate-esta-quarta/
Le Cordon Bleu: chance de bolsa até esta quarta
https://lulacerda.ig.com.br/le-cordon-bleu-chance-de-bolsa-ate-esta-quarta/
100% [...........................

IndexError: list index out of range

In [ ]:
noticias[0]